In [1]:
import os
import pandas as pd
import numpy as np
from azureml.core import Workspace, Dataset

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (ruamel.yaml 0.17.9 (c:\users\klim_\anaconda3\lib\site-packages), Requirement.parse('ruamel.yaml<0.17.5,>=0.15.35'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (ruamel.yaml 0.17.9 (c:\users\klim_\anaconda3\lib\site-packages), Requirement.parse('ruamel.yaml<0.17.5,>=0.15.35'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (ruamel.yaml 0.17.9 (c:\users\klim_\anaconda3\lib\site-packages), Requirement.parse('ruamel.yaml<0.17.5,>=0.15.35'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run

In [2]:
WORKDIR = os.getcwd()

In [3]:
# Create a folder
BATCH_FOLDER = 'batch-data'
os.makedirs(BATCH_FOLDER, exist_ok=True)
print("Folder created!")

Folder created!


In [4]:
# get the workspace from config.json
ws = Workspace.from_config()
# get the datastore to upload our data
datastore = ws.get_default_datastore()

In [7]:
# retrieve test dataset from Azure Datastore
ds = Dataset.get_by_name(ws, name="processed_weather_data")
df = ds.to_pandas_dataframe()
df.head()

,data_index_,temperature,solar_ghi,solar_prediction_mw,wind_prediction_mw,load_actuals_mw
0,2020-01-01 00:00:00,274.989655,0.0,0.0,70.865426,95.756328
1,2020-01-01 00:15:00,274.925659,0.0,0.0,69.296785,94.836196
2,2020-01-01 00:30:00,274.861694,0.0,0.0,66.977409,93.798127
3,2020-01-01 00:45:00,274.797699,0.0,0.0,64.305715,92.162902
4,2020-01-01 01:00:00,274.423157,0.0,0.0,61.128262,91.506670


In [13]:
# pick up last week, drop target and timestamp, convert to numpy array
X = df.iloc[-7:]
X = X.drop(['load_actuals_mw', 'data_index_'], axis=1)
X = X.to_numpy()

In [16]:
# Save each sample as a separate file
print("Saving files...")
for i in range(len(X)):
    fname = str(i+1) + '.csv'
    X[i].tofile(os.path.join(BATCH_FOLDER, fname), sep=",")
print("files saved!")

Saving files...
files saved!


In [17]:
datastore.upload(src_dir="batch-data", target_path="batch-data", overwrite=True, show_progress=True)

Uploading an estimated of 7 files
Uploading batch-data\1.csv
Uploaded batch-data\1.csv, 1 files out of an estimated total of 7
Uploading batch-data\2.csv
Uploaded batch-data\2.csv, 2 files out of an estimated total of 7
Uploading batch-data\5.csv
Uploaded batch-data\5.csv, 3 files out of an estimated total of 7
Uploading batch-data\3.csv
Uploaded batch-data\3.csv, 4 files out of an estimated total of 7
Uploading batch-data\4.csv
Uploaded batch-data\4.csv, 5 files out of an estimated total of 7
Uploading batch-data\6.csv
Uploaded batch-data\6.csv, 6 files out of an estimated total of 7
Uploading batch-data\7.csv
Uploaded batch-data\7.csv, 7 files out of an estimated total of 7
Uploaded 7 files


$AZUREML_DATAREFERENCE_c30e69e832ba49dbb8d0319b0208a4be

In [18]:
# Register a dataset for the input data
batch_data_set = Dataset.File.from_files(path=(datastore, 'batch-data/'), validate=False)
try:
    batch_data_set = batch_data_set.register(workspace=ws, 
                                             name='batch-data',
                                             description='batch data for pytown demand energy forecast',
                                             create_new_version=True)
except Exception as ex:
    print(ex)

print("Done!")

Done!
